In [46]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [2]:
# define the MSEIRS function
def MSEIRS_model(time, new_born_w_immunity, current_state, beta, sigma, gamma, delta, lam):
    M, S, E, I, R = current_state
    
    N = M + S + E + I + R
    # -weakend rate * infant with immunity + new born rate * new born with immunity prob
    dM = -delta * M + lam * I * new_born_w_immunity
    # - contact rate * contact probabilities 
    # + new born losing passive immunity rate 
    # + new born rate * new born without immunity prob
    # + population losing immunity
    dS = -(beta * S * I) / N + delta * M + lam * I * (1-new_born_w_immunity)
    dE = (beta * S * I) / N - sigma * E
    # infectious rate - recovery rate - death rate
    dI = sigma * E - gamma * I - lam * I
    # recovery rate
    dR = gamma * I
    
    return [dM, dS, dE, dI, dR]

times = np.arange(0, 366)

In [3]:
# initialise parameters
params = {
    'delta': 0.1,
    'beta': 0.5,
    'sigma': 0.25,
    'gamma': 0.2,
    'lam': 0.05
}

In [17]:
maternal_params = np.arange(0,1.1,0.2)
summary_names = ['M', 'S', 'E', 'I', 'R']
l = []
for p in maternal_params:
    p = round(p,2)
    # initialise the initialise state of MSEIRS
    initial_state = [0, 999999, 1, 0, 0]
    # Solve the MSEIRS model using solve_ivp
    solution = solve_ivp(lambda t, y: MSEIRS_model(t, p, y, **params), [times[0], times[-1]], initial_state, t_eval=times)
    for idx, i in enumerate(solution.y):
        test = i
        for idx2, j in enumerate(i):
            l.append([p, summary_names[idx], idx2, j])

df = pd.DataFrame(data=l)
df = df.rename({0: 'p', 1: 'state', 2: 'step', 3: 'val'}, axis=1)
df.head()

,p,state,step,val
0,0.0,M,0,0.0
1,0.0,M,1,0.0
2,0.0,M,2,0.0
3,0.0,M,3,0.0
4,0.0,M,4,0.0


In [88]:
df[df['state'] == 'M'].groupby(['p', 'state'])['val'].describe()

,,count,mean,std,min,25%,50%,75%,max
p,state,,,,,,,,
0.0,M,366.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
0.2,M,366.0,1039.952888,2049.689096,0.0,0.287676,15.868986,736.511716,7540.010336
0.4,M,366.0,2061.770639,4051.282534,0.0,0.650993,33.302095,1486.314563,14895.020926
0.6,M,366.0,3066.241593,6006.441892,0.0,1.110542,53.188280,2249.512351,22070.805978
0.8,M,366.0,4054.122115,7916.748123,0.0,1.678503,74.749443,3025.557369,29072.761537
1.0,M,366.0,5026.138054,9783.705573,0.0,2.378514,99.162046,3788.504329,35906.216924


In [6]:
81913.170546/87068.285411 * 100

94.07922776856621

In [78]:
fig = px.line(df[df['state'] == 'M'], x='step',  y='val', color='p', template='plotly_white',
              color_discrete_sequence=["blue", '#a3a3a3', '#bbbbbb', '#cccccc', '#dddddd', 'red'])
fig.update_layout(
    autosize=False,
    width=500,
    height=500,
    margin=dict(
        l=50,
        r=50,
        b=80,
        t=80,
        pad=4
    ),
    yaxis=dict(
        title_text="Population",
    ),
    xaxis=dict(
        title_text="Day",
    ),
    title={
    'text': "Effect of p on Maternally-derived Immunity Population",
    'y':0.9,
    'x':0.5,
    'xanchor': 'center',
    'yanchor': 'top'},
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="right",
        x=0.99
    )
)
fig.show()
fig.write_image('maternal_immunity_M.png')

In [79]:
fig = px.line(df[df['state'] == 'S'], x='step',  y='val', color='p', template='plotly_white',
              color_discrete_sequence=["blue", '#a3a3a3', '#bbbbbb', '#cccccc', '#dddddd', 'red'])
fig.update_layout(
    autosize=False,
    width=500,
    height=500,
    margin=dict(
        l=50,
        r=50,
        b=80,
        t=80,
        pad=4
    ),
    yaxis=dict(
        title_text="Population",
    ),
    xaxis=dict(
        title_text="Day",
    ),
    title={
    'text': "Effect of p on Susceptible Population",
    'y':0.9,
    'x':0.5,
    'xanchor': 'center',
    'yanchor': 'top'},
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="right",
        x=0.99
    )
)
fig.show()
fig.write_image('maternal_immunity_S.png')

In [80]:
fig = px.line(df[df['state'] == 'E'], x='step',  y='val', color='p', template='plotly_white',
              color_discrete_sequence=["blue", '#a3a3a3', '#bbbbbb', '#cccccc', '#dddddd', 'red'])
fig.update_layout(
    autosize=False,
    width=500,
    height=500,
    margin=dict(
        l=50,
        r=50,
        b=80,
        t=80,
        pad=4
    ),
    yaxis=dict(
        title_text="Population",
    ),
    xaxis=dict(
        title_text="Day",
    ),
    title={
    'text': "Effect of p on Exposed Population",
    'y':0.9,
    'x':0.5,
    'xanchor': 'center',
    'yanchor': 'top'},
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="right",
        x=0.99
    )
)
fig.show()
fig.write_image('maternal_immunity_E.png')

In [81]:
fig = px.line(df[df['state'] == 'I'], x='step',  y='val', color='p', template='plotly_white',
              color_discrete_sequence=["blue", '#a3a3a3', '#bbbbbb', '#cccccc', '#dddddd', 'red'])
fig.update_layout(
    autosize=False,
    width=500,
    height=500,
    margin=dict(
        l=50,
        r=50,
        b=80,
        t=80,
        pad=4
    ),
    yaxis=dict(
        title_text="Population",
    ),
    xaxis=dict(
        title_text="Day",
    ),
    title={
    'text': "Effect of p on Infectious Population",
    'y':0.9,
    'x':0.5,
    'xanchor': 'center',
    'yanchor': 'top'},
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="right",
        x=0.99
    )
)
fig.show()
fig.write_image('maternal_immunity_I.png')

In [82]:
fig = px.line(df[df['state'] == 'I'], x='step',  y='val', color='p', template='plotly_white',
              color_discrete_sequence=["blue", '#a3a3a3', '#bbbbbb', '#cccccc', '#dddddd', 'red'])
fig.update_layout(
    autosize=False,
    width=500,
    height=500,
    margin=dict(
        l=50,
        r=50,
        b=80,
        t=80,
        pad=4
    ),
    yaxis=dict(
        title_text="Population",
    ),
    xaxis=dict(
        title_text="Day",
    ),
    title={
    'text': "Effect of p on Recovered Population",
    'y':0.9,
    'x':0.5,
    'xanchor': 'center',
    'yanchor': 'top'},
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="right",
        x=0.99
    )
)
fig.show()
fig.write_image('maternal_immunity_R.png')